# DTI Probabilistic Tractography

In the previous tutorial we covered a simple example of determinisc tractography. The approach described is fast and very straightforward, however there are several disadvantages. In particular, deterministic DTI tractography can be overly simplistic, as it does not account for crossing, diverging, or converging fibers within a voxel. It is also highly sensitive to noise and partial volume effects, which can lead to erroneous tracking results.

Probabilistic tractography models the uncertainty in fiber direction estimation and generates multiple possible pathways for each seed point, effectively sampling from a distribution of possible directions. The output is a distribution of possible pathways from each seed point, resulting in a density map or connectivity map that represents the probability of connection between regions. This approach can better capture the complexity of the brain's white matter, including crossing fibers and regions with uncertain fiber direction. It is also more robust to noise and partial volume effects. 

However, probabilistic tractography is computationally more intensive than deterministic tractography and the results can be harder to interpret due to the probabilistic nature of the output. This technique will also generate a high rate of false-positive connections, requiring careful analysis, including thresholding.

In this tutorial we will cover how to perform probabilistic fibre tracking from the orientation distribution estimated from DTI. 

We will start by loading the pre-processed diffusion MRI data:

In [2]:
#load general modules
import os
import numpy as np
import nibabel as nib

#load dipy modules
from dipy.core.gradients import gradient_table
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, save_nifti

#load modules for visualization
import matplotlib.pyplot as plt

#define the paths to the data
scripts_dir = os.getcwd()
bids_dir = f"{scripts_dir[0:61]}/data/bids"
out_dir = f"{scripts_dir[0:61]}/data/derivatives"

#subject code to be used in this example
sub='01'

#load the pre-processed data
data_preproc, affine, data_preproc_img = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.nii.gz", return_img=True)

#load the bvals and bvecs
bvals, bvecs = read_bvals_bvecs(f"{bids_dir}/sub-{sub}/dwi/sub-{sub}_acq-AP_dwi.bval", f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.eddy_rotated_bvecs")

#select data for DTI model fitting
#create a mask bo b-values less than 1300 s/mm2
bval_mask = bvals < 1300

#select the data for DTI model fitting
data_for_dti = data_preproc[..., bval_mask]

bvals_for_dti = bvals[bval_mask]
bvecs_for_dti = bvecs[bval_mask, :]

#load the gradient table
gtab_for_dti = gradient_table(bvals_for_dti, bvecs_for_dti)


#load the binary brain mask
brain_mask, affine_mask = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/hifi_nodif_brain_mask.nii.gz")
masked_data = data_for_dti * brain_mask[..., np.newaxis]

### Step 1: Fit the DTI model and generate the orientation distribution function

For this step we need to generate the orientation distribution function (ODF) frim the DTI model. Firstly we fit the DTI model to the data, and then we calculate the ODF using the dtifit atribute odf. For this we need to input directions evenly sampling a 3D sphere, which can be loaded from dipy.data. 


In [3]:
#import the TensorModel from dipy
from dipy.reconst.dti import TensorModel
from dipy.data import get_sphere

#fit the DTI model
dtimodel = TensorModel(gtab_for_dti)
dtifit = dtimodel.fit(data_for_dti, mask=brain_mask) 

#load the directions evenly distributed on a sphere
sphere = get_sphere('symmetric724')

#generate the ODF
tensor_odfs = dtifit.odf(sphere)


For quality assurance, we can visualise the ODF for each voxel using the fury package:

In [ ]:
from dipy.viz import window, actor

scene = window.Scene()

odf_actor = actor.odf_slicer(tensor_odfs, sphere=sphere, scale=0.5,
                             colormap='plasma')
scene.add(odf_actor)
print('Saving illustration as tensor_odfs.png')
window.record(scene, n_frames=1, out_path=f"{out_dir}/DTI/sub-{sub}/tensor_odfs.png", size=(600, 600))
window.show(scene)

### Steps 2 & 3: Define the seeds and stopping criteria

We will use the same seed mask and stopping criteria as for the deterministic tracking example. We seed the voxels in a corpus callosum sagittal slice with a grid density of 2 × 2 × 2. Streamlines will stop propagating when entering voxels with FA values lower than 0.2.

In [4]:
from dipy.tracking import utils
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

#load corpus callosum mask
cc_mask, affine_cc = load_nifti(f"{out_dir}/DTI/sub-{sub}/CC_mask.nii.gz")

#create the seed mask
seed_mask = (cc_mask == 1)
seeds = utils.seeds_from_mask(seed_mask, affine_cc, density=[2, 2, 2])

#calculate fa map and define stopping criterion
fa = dtifit.fa
stopping_criterion = ThresholdStoppingCriterion(fa, 0.2)

### Step 4: Propagating the streamlines

The probabilistic streamlines will be generated using LocalTracking, similarly to the deterministic tractography example. However, there is one additional step: we need to instantiate a "ProbabiticDirectionGetter" class object in order to use DTI's ODFs for probabilistic tracking.

Note: Input variable "max_angle" gives the maximum allowed angle between the incoming direction and the new direction in tractography propagation.

In [5]:
from dipy.direction import ProbabilisticDirectionGetter

#instanciate the probabilistic direction getter
pmf = tensor_odfs.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
                                                sphere=sphere)

Finally, we initalise the streamlines generator, create the streamlines and dave them as a Trackvis file:

In [6]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

#initialise the streamlines generator
streamlines_generator = LocalTracking(prob_dg, stopping_criterion, seeds,
                                      affine=affine, step_size=0.5)

#generate the streamlines
streamlines = Streamlines(streamlines_generator)

#save the streamlines
sft = StatefulTractogram(streamlines, data_preproc_img, Space.RASMM)
save_trk(sft, f"{out_dir}/DTI/sub-{sub}/dti_CC_probabilistic_tract.trk", streamlines)

The output can be visualised with the fury package:

In [ ]:
from dipy.viz import window, actor, has_fury
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path=f"{out_dir}/DTI/sub-{sub}/dti_tractogram_prob.png", size=(800, 800))
    window.show(scene)